<a href="https://colab.research.google.com/github/veradureke/Deforestation-Exploration/blob/main/SpaceLab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
! apt-get install git


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.10).
0 upgraded, 0 newly installed, 0 to remove and 16 not upgraded.


In [1]:
!git clone https://github.com/spacelabdev/new_world_disco.git

Cloning into 'new_world_disco'...
remote: Enumerating objects: 527, done.
remote: Counting objects: 100% (124/124), done.
remote: Compressing objects: 100% (93/93), done.
remote: Total 527 (delta 69), reused 62 (delta 31), pack-reused 403
Receiving objects: 100% (527/527), 36.81 MiB | 32.19 MiB/s, done.
Resolving deltas: 100% (311/311), done.


In [ ]:
!tar -xvf "/content/exonet_inputs/test_centroids.tar.gz" -C "/content/exonet_inputs"

In [ ]:
!tar -xvf "/content/exonet_inputs/test_info.tar.gz" -C "/content/exonet_inputs"

In [ ]:
!tar -xvf "/content/exonet_inputs/test_lightcurves.tar.gz" -C "/content/exonet_inputs"

In [ ]:
!tar -xvf "/content/exonet_inputs/train_centroids.tar.gz" -C "/content/exonet_inputs"

In [ ]:
!tar -xvf "/content/exonet_inputs/train_lightcurves.tar.gz" -C "/content/exonet_inputs"

In [ ]:
!unzip /content/exonet_inputs/train_info.zip

In [ ]:
!tar -xvf "/content/exonet_inputs/val_centroids.tar.gz" -C "/content/exonet_inputs"

In [ ]:
!tar -xvf "/content/exonet_inputs/val_lightcurves.tar.gz" -C "/content/exonet_inputs"

In [ ]:
import os
import numpy as np
import glob as glob
from tqdm import tqdm
from random import random
import argparse
import pandas as pd
import pdb

In [ ]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torch.nn as nn

In [ ]:
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score


In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec


In [ ]:
    def __init__(self, filepath):

        ### list of global, local, and info files (assumes certain names of files)
        self.flist_global = np.sort(glob.glob(os.path.join(filepath, '*global.npy')))
        self.flist_local = np.sort(glob.glob(os.path.join(filepath, '*local.npy')))
        self.flist_info = np.sort(glob.glob(os.path.join(filepath, '*info.npy')))

        ### list of whitened centroid files
        self.flist_global_cen = np.sort(glob.glob(os.path.join(filepath, '*global_cen_w.npy')))
        self.flist_local_cen = np.sort(glob.glob(os.path.join(filepath, '*local_cen_w.npy')))

        ### ids = {TIC}_{TCE}
        self.ids = np.sort([(x.split('/')[-1]).split('_')[1] + '_' + (x.split('/')[-1]).split('_')[2] for x in self.flist_global])

    def __len__(self):

        return self.ids.shape[0]

    def __getitem__(self, idx):

        ### grab local and global views
        data_global = np.load(self.flist_global[idx])
        data_local = np.load(self.flist_local[idx])

        ### grab centroid views
        data_global_cen = np.load(self.flist_global_cen[idx])
        data_local_cen = np.load(self.flist_local_cen[idx])

        ### info file contains: [0]kic, [1]tce, [2]period, [3]epoch, [4]duration, [5]label)
        data_info = np.load(self.flist_info[idx])

        return (data_local, data_global, data_local_cen, data_global_cen, data_info[6:]), data_info[5]